In [ ]:
!pip install gradio

In [30]:
import numpy as np
import gradio as gr
import torch
import matplotlib.pyplot as plt
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

#Loading the Model

In [31]:
model = None  # Initialize model variable
try:
    # Load the model from the specified path
    model = AutoModelForSequenceClassification.from_pretrained("/content/contentsaved_model_bert_sentiment")
    model_weights = torch.load("//content/contentsaved_model_bert_sentiment/model.safetensors")
    model.load_state_dict(model_weights)
except Exception as e:
    print(f"Error loading model: {str(e)}")  # Print error message if loading fails

Error loading model: Weights only load failed. In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
Please file an issue with the following so that we can make `weights_only=True` compatible with your use case: WeightsUnpickler error: Unsupported operand 232

Check the documentation of torch.load to learn more about types accepted by default with weights_only https://pytorch.org/docs/stable/generated/torch.load.html.


# Load the tokenizer

In [32]:
tokenizer = AutoTokenizer.from_pretrained("/content/contentsaved_model_bert_sentiment")

# Set up the sentiment analysis pipeline

In [33]:
if model is not None:
    sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
else:
    sentiment_pipeline = None  # Set pipeline to None if model is not loaded

Device set to use cpu


#Review Classification Function with Plot

In [40]:
def classify_review_with_plot(review):
    if sentiment_pipeline is None:
        return "Error: Sentiment analysis model is not loaded.", None  # Error message if model is not loaded

    try:
        result = sentiment_pipeline(review)  # Analyze sentiment for the review
        label = result[0]['label']  # Get the label from the model
        score = result[0]['score']  # Confidence score

        # Create a mapping for labels
        label_mapping = {
            "LABEL_0": "NEGATIVE",  # Assuming LABEL_0 corresponds to negative sentiment
            "LABEL_1": "NEUTRAL",   # Assuming LABEL_1 corresponds to neutral sentiment
            "LABEL_2": "POSITIVE"    # Assuming LABEL_2 corresponds to positive sentiment
        }

        # Map the label to a human-readable format
        human_readable_label = label_mapping.get(label, "UNKNOWN")  # Default to "UNKNOWN" if label is not found

        # Prepare values for the bar chart
        labels = ["Positive", "Negative", "Neutral"]
        values = [0, 0, 0]  # Initialize values for each category

        # Set values based on the mapped label
        if human_readable_label == "POSITIVE":
            values[0] = score
        elif human_readable_label == "NEGATIVE":
            values[1] = score
        elif human_readable_label == "NEUTRAL":
            values[2] = score

        # Create a bar chart
        fig, ax = plt.subplots()
        ax.bar(labels, values, color=["green", "red", "blue"])
        ax.set_ylabel("Confidence Score")
        ax.set_title("Review Classification")

        # Return the sentiment label and score
        sentiment_result = f"Sentiment: {human_readable_label}, Score: {score:.2f}"
        return sentiment_result, fig
    except Exception as e:
        return f"Error: {str(e)}", None  # Error message if an exception occurs

# Set up Gradio interface

In [41]:
with gr.Blocks() as app:
    gr.Markdown("## Review Analysis Application")  # Application title

    with gr.Tab("Review Classification"):  # Review classification tab
        review_input = gr.Textbox(label="Enter your review here", placeholder="Type your review here...")  # Review input field
        classify_button = gr.Button("Classify")  # Classification button
        classification_output = gr.Textbox(label="Classification Result")  # Classification result output field
        chart_output = gr.Plot(label="Graphical Results")  # Graphical results output field

        # Link the button to the function
        classify_button.click(classify_review_with_plot, inputs=review_input, outputs=[classification_output, chart_output])


# Launch the application

In [42]:
app.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c7f4869b820d2753d0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
